In [ ]:
from coincidence_matching import match_parts, get_export_id_types, match_parts_dict
from zipfile import ZipFile
import json
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from automate import Part
import numpy as np
import meshplot as mp
from collections import Counter
import torch

from apikey.onshape import Onshape

from brepmatching.data import BRepMatchingDataset

import os
from pathlib import Path
str(Path.home().joinpath('.config','onshapecreds.json'))

api = Onshape(stack='https://cad.onshape.com', creds=str(Path.home().joinpath('.config','onshapecreds.json')), logging=False)


def dowload_part(did, mv, eid):
    response = api.request(
        method='get', 
        path=f'/api/partstudios/d/{did}/m/{mv}/e/{eid}/parasolid', query={'includeExportIds':True})


In [ ]:
from coincidence_matching import match_parts, get_export_id_types, match_parts_dict
from zipfile import ZipFile
import json
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from automate import Part
import numpy as np
import meshplot as mp
from collections import Counter
import torch
from automate import Part
from brepmatching.data import BRepMatchingDataset

In [ ]:
from zipfile import ZipFile
import pandas as pd
from tqdm import tqdm
from apikey.onshape import Onshape
from pathlib import Path

In [ ]:
with ZipFile('../data/TopoandGeoV2FullRunWith100SamplesBaseline.zip','r') as zf:
    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)
    variations[:100]
    variations = variations[variations.fail == 0]

In [ ]:
primary_orig_path = 'data/BrepsWithReference/' + variations.iloc[0].ps_orig
primary_var_path = 'data/BrepsWithReference/' + variations.iloc[0].ps_var
primary_match_path = 'data/Matches/' + variations.iloc[0].matchFile

baseline_orig_path = '../data/benTestBrep1.x_t'
baseline_var_path = '../data/benTestBrep2.x_t'
baseline_match_path = '../data/benTestMatch.json'

In [ ]:
with ZipFile('../data/TopoandGeoV2FullRunWith100SamplesBaseline.zip','r') as zf:
    with zf.open(primary_orig_path,'r') as f:
        primary_orig_data = f.read().decode('utf-8')
    with zf.open(primary_var_path,'r') as f:
        primary_var_data = f.read().decode('utf-8')
    with zf.open(primary_match_path,'r') as f:
        primary_match = json.load(f)

primary_orig_part = Part(primary_orig_data)
primary_var_part = Part(primary_var_data)

baseline_orig_part = Part(baseline_orig_path)
baseline_var_part = Part(baseline_var_path)
with open(baseline_match_path, 'r') as f:
    baseline_match = json.load(f)

primary_orig_export_ids = get_export_id_types(primary_orig_data)
primary_var_export_ids = get_export_id_types(primary_var_data)

baseline_orig_export_ids = get_export_id_types(baseline_orig_path)
baseline_var_export_ids = get_export_id_types(baseline_var_path)

orig_exact_matching = match_parts(baseline_orig_path, primary_orig_data, True)
var_exact_matching = match_parts(baseline_var_path, primary_var_data, True)

orig_exact_matching_dict = match_parts_dict(baseline_orig_path, primary_orig_data, True)
var_exact_matching_dict = match_parts_dict(baseline_var_path, primary_var_data, True)


In [ ]:
types_to_match = ['PK_CLASS_face', 'PK_CLASS_edge', 'PK_CLASS_vertex']
type_counts = np.zeros((3,))
translated_matching = {}
for k, match in baseline_match.items():
    orig_export_id = match['val1']
    var_export_id = match['val2']
    assert(baseline_orig_export_ids[orig_export_id] == baseline_var_export_ids[var_export_id])
    if baseline_var_export_ids[var_export_id] in types_to_match:
        t = types_to_match.index(baseline_var_export_ids[var_export_id])
        type_counts[t] = type_counts[t] + 1
        new_var_export_id = var_exact_matching_dict[var_export_id]
        translated_matching[k] = {'val1':orig_export_id, 'val2':new_var_export_id}

In [ ]:
type_counts